In [2]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import joblib
from datasist.structdata import detect_outliers

In [11]:
df = pd.read_excel('D:/Courses/AI/Epsilon AI/Session 30/Assignment/Assignment/Airflight_prices.xlsx')
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [13]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [16]:
df.corr()

,price
price,1.0


In [17]:
df.dropna(axis=0,inplace=True)

In [18]:
df.isnull().sum()

airline            0
date_of_journey    0
source             0
destination        0
route              0
dep_time           0
arrival_time       0
duration           0
total_stops        0
additional_info    0
price              0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   airline          10682 non-null  object
 1   date_of_journey  10682 non-null  object
 2   source           10682 non-null  object
 3   destination      10682 non-null  object
 4   route            10682 non-null  object
 5   dep_time         10682 non-null  object
 6   arrival_time     10682 non-null  object
 7   duration         10682 non-null  object
 8   total_stops      10682 non-null  object
 9   additional_info  10682 non-null  object
 10  price            10682 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1001.4+ KB


In [20]:
df.columns=df.columns.str.lower()
df.columns

Index(['airline', 'date_of_journey', 'source', 'destination', 'route',
       'dep_time', 'arrival_time', 'duration', 'total_stops',
       'additional_info', 'price'],
      dtype='object')

In [21]:
num_col=df.select_dtypes('number').columns
num_col

Index(['price'], dtype='object')

In [22]:
df.drop(['duration','arrival_time','dep_time','date_of_journey','additional_info'], axis=1 ,inplace=True)
df

,airline,source,destination,route,total_stops,price
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,3897
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,7662
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,13882
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,6218
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,13302
...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,CCU → BLR,non-stop,4107
10679,Air India,Kolkata,Banglore,CCU → BLR,non-stop,4145
10680,Jet Airways,Banglore,Delhi,BLR → DEL,non-stop,7229
10681,Vistara,Banglore,New Delhi,BLR → DEL,non-stop,12648


In [23]:
cat_col=df.select_dtypes('object').columns
cat_col

Index(['airline', 'source', 'destination', 'route', 'total_stops'], dtype='object')

In [24]:
df=pd.get_dummies(df,cat_col,drop_first=True)
df

,price,airline_Air India,airline_GoAir,airline_IndiGo,airline_Jet Airways,airline_Jet Airways Business,airline_Multiple carriers,airline_Multiple carriers Premium economy,airline_SpiceJet,airline_Trujet,...,route_DEL → NAG → BOM → COK,route_DEL → PNQ → COK,route_DEL → RPR → NAG → BOM → COK,route_DEL → TRV → COK,route_DEL → UDR → BOM → COK,route_MAA → CCU,total_stops_2 stops,total_stops_3 stops,total_stops_4 stops,total_stops_non-stop
0,3897,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,7662,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,13882,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,6218,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13302,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,4107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10679,4145,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10680,7229,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10681,12648,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [25]:
df.corr()

,price,airline_Air India,airline_GoAir,airline_IndiGo,airline_Jet Airways,airline_Jet Airways Business,airline_Multiple carriers,airline_Multiple carriers Premium economy,airline_SpiceJet,airline_Trujet,...,route_DEL → NAG → BOM → COK,route_DEL → PNQ → COK,route_DEL → RPR → NAG → BOM → COK,route_DEL → TRV → COK,route_DEL → UDR → BOM → COK,route_MAA → CCU,total_stops_2 stops,total_stops_3 stops,total_stops_4 stops,total_stops_non-stop
price,1.000000,0.050432,-0.095151,-0.361070,0.416124,0.253303,0.139793,0.017650,-0.296565,-0.010381,...,0.057309,-0.033859,0.015621,-0.010512,0.042516,-0.179223,0.320508,0.056769,0.018043,-0.613801
airline_Air India,0.050432,1.000000,-0.060221,-0.215977,-0.332324,-0.010497,-0.157223,-0.015456,-0.127510,-0.004284,...,-0.005099,-0.027819,0.090168,0.074394,0.049628,-0.051068,0.366871,0.115659,0.021852,-0.083698
airline_GoAir,-0.095151,-0.060221,1.000000,-0.066339,-0.102076,-0.003224,-0.048292,-0.004747,-0.039166,-0.001316,...,-0.011125,-0.008545,-0.005430,-0.005268,-0.005588,-0.026156,-0.055396,-0.008846,-0.001316,0.042744
airline_IndiGo,-0.361070,-0.215977,-0.066339,1.000000,-0.366085,-0.011563,-0.173196,-0.017026,-0.140464,-0.004720,...,-0.039899,0.014912,-0.019474,-0.006605,-0.020040,0.141911,-0.185752,-0.031726,-0.004720,0.288753
airline_Jet Airways,0.416124,-0.332324,-0.102076,-0.366085,1.000000,-0.017793,-0.266497,-0.026199,-0.216132,-0.007262,...,0.084992,-0.047154,-0.029965,-0.029069,0.002444,-0.144341,0.079987,-0.048816,-0.007262,-0.263941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
route_MAA → CCU,-0.179223,-0.051068,-0.026156,0.141911,-0.144341,-0.004559,-0.068288,-0.006713,0.187590,-0.001861,...,-0.015732,-0.012083,-0.007678,-0.007449,-0.007901,1.000000,-0.078334,-0.012509,-0.001861,0.276021
total_stops_2 stops,0.320508,0.366871,-0.055396,-0.185752,0.079987,0.012964,-0.108085,-0.014218,-0.117294,-0.003941,...,0.200828,-0.025591,-0.016262,-0.015776,0.100867,-0.078334,1.000000,-0.026493,-0.003941,-0.283796
total_stops_3 stops,0.056769,0.115659,-0.008846,-0.031726,-0.048816,-0.001542,0.013576,-0.002270,-0.018730,-0.000629,...,-0.005320,-0.004086,0.613829,-0.002519,-0.002672,-0.012509,-0.026493,1.000000,-0.000629,-0.045319
total_stops_4 stops,0.018043,0.021852,-0.001316,-0.004720,-0.007262,-0.000229,-0.003436,-0.000338,-0.002786,-0.000094,...,-0.000791,-0.000608,-0.000386,-0.000375,-0.000398,-0.001861,-0.003941,-0.000629,1.000000,-0.006742


In [26]:
x=df.drop('price',axis=1)
y=df['price']

In [27]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [28]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [29]:
models={
    'LR':LinearRegression(),
    'KNN':KNeighborsRegressor(n_neighbors=7),
    'SVR':SVR()
}

In [31]:
for name,model in models.items():
    print('using:',name)
    model.fit(x_train_scaled,y_train)
    print('train accuracy: ',model.score(x_train_scaled,y_train))
    print('test accuracy: ',model.score(x_test_scaled,y_test))
    print('-'*20)

using: LR
train accuracy:  0.634194798515035
test accuracy:  -9.206629454941224e+24
--------------------
using: KNN
train accuracy:  0.6210359371955728
test accuracy:  0.6183478848659518
--------------------
using: SVR
train accuracy:  0.024868368799083163
test accuracy:  0.029132940855224995
--------------------


In [32]:
features=x.columns.to_list()
features

['airline_Air India',
 'airline_GoAir',
 'airline_IndiGo',
 'airline_Jet Airways',
 'airline_Jet Airways Business',
 'airline_Multiple carriers',
 'airline_Multiple carriers Premium economy',
 'airline_SpiceJet',
 'airline_Trujet',
 'airline_Vistara',
 'airline_Vistara Premium economy',
 'source_Chennai',
 'source_Delhi',
 'source_Kolkata',
 'source_Mumbai',
 'destination_Cochin',
 'destination_Delhi',
 'destination_Hyderabad',
 'destination_Kolkata',
 'destination_New Delhi',
 'route_BLR → BBI → DEL',
 'route_BLR → BDQ → DEL',
 'route_BLR → BOM → AMD → DEL',
 'route_BLR → BOM → BHO → DEL',
 'route_BLR → BOM → DEL',
 'route_BLR → BOM → IDR → DEL',
 'route_BLR → BOM → IDR → GWL → DEL',
 'route_BLR → BOM → IXC → DEL',
 'route_BLR → BOM → JDH → DEL',
 'route_BLR → BOM → NAG → DEL',
 'route_BLR → BOM → UDR → DEL',
 'route_BLR → CCU → BBI → DEL',
 'route_BLR → CCU → BBI → HYD → DEL',
 'route_BLR → CCU → BBI → HYD → VGA → DEL',
 'route_BLR → CCU → DEL',
 'route_BLR → CCU → GAU → DEL',
 'rout

In [33]:
joblib.dump(model,'Air_model.h5')
joblib.dump(scaler,'Air_scaler.h5')
joblib.dump(features,'AIr_features.h5')

['AIr_features.h5']